In [104]:
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline
import numpy as np
import numexpr as ne
from datetime import datetime
import ipdb
# from scipy.ndimage import imread
%load_ext line_profiler
import import_ipynb
from mathfunk import *

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [113]:
class Hexgrid():
    '''Simulates a turbidity current using a CA. '''
    def __init__(self, Nx, Ny, reposeAngle = np.deg2rad(0), dx=1, terrain=None):
        ################ Constants ######################
        self.g = 9.81 # Gravitational acceleration
        self.f = 0.04 # Darcy-Weisbach coeff
        self.a = 0.43 # Empirical coefficient
        self.rho_a = 0.5 # ambient density
        self.rho_j = np.array([1]) # List of current densities
        self.Nj = 1 # Number of sediment types
        
            # Constants used in I_1:
        self.p_f = np.deg2rad(1) # Height threshold friction angle
        self.p_adh = 1
        ############## Input variables ###################
        self.Nx = Nx
        self.Ny = Ny
        self.dx = dx
        self.reposeAngle = reposeAngle
        
        ################     Grid       ###################
        self.X = np.zeros((Ny, Nx, 2)) # X[:,:,0] = X coords, X[:,:,1] = Y coords
        for j in range(Ny):
            self.X[j,:,0] = j*dx/2 + np.arange(Nx) * dx    
            self.X[j,:,1] = -np.ones(Nx) * dx*np.sqrt(3)/2 * j

        
        ################# Cell substate storage ####################        
#         self.Q_a   = np.zeros((self.Ny,self.Nx)) # Cell altitude (bathymetry at t = 0)
        self.Q_th  = np.zeros((self.Ny,self.Nx)) # Turbidity current thickness
        self.Q_v   = np.zeros((self.Ny,self.Nx)) # Turbidity current speed (scalar)
        self.Q_cj  = np.zeros((self.Ny,self.Nx,self.Nj)) # jth current sediment volume concentration
        self.Q_cbj = np.zeros((self.Ny,self.Nx,self.Nj)) # jth bed sediment volume fraction
        self.Q_d   = np.ones((self.Ny,self.Nx)) * np.inf # Thickness of soft sediment
        self.Q_d[1:self.Ny-1,1:self.Nx-1] = 0
        self.Q_a = self.Q_d.copy()
        self.Q_o   = np.zeros((self.Ny,self.Nx,6)) # Density current outflow
        
        ################### Set Initial conditions #####################
        self.CellArea = calc_hexagon_area(dx)
        self.setBathymetry(terrain)
        self.diff = np.zeros((self.Ny-2,self.Ny-2,6))
        self.seaBedDiff = np.zeros((self.Ny-2,self.Nx-2,6))
        self.calc_bathymetryDiff()
        
        self.totalheight = self.Q_d + self.Q_a
        
        self.defineNeighbors()
                                
        ################################################################
        ##########################  Methods ############################
        ################################################################
    
    def defineNeighbors(self):
        ''' 
        This function defines indices that can be used to reference the neighbors of a cell.\
        Use: self.Q_v[self.NEIGHBOR[0]] = NW neighbors' value of Q_v
        '''
        self.NEIGHBOR = []
        self.NEIGHBOR.append( np.ix_(np.arange(0,self.Ny-2), np.arange(1,self.Nx-1)))
        self.NEIGHBOR.append( np.ix_(np.arange(0,self.Ny-2), np.arange(2,self.Nx))  )
        self.NEIGHBOR.append( np.ix_(np.arange(1,self.Ny-1), np.arange(2,self.Nx))  )
        self.NEIGHBOR.append( np.ix_(np.arange(2,self.Ny),   np.arange(1,self.Nx-1)))
        self.NEIGHBOR.append( np.ix_(np.arange(2,self.Ny),   np.arange(0,self.Nx-2)))                     
        self.NEIGHBOR.append( np.ix_(np.arange(1,self.Ny-1), np.arange(0,self.Nx-2)))
        
    def time_step(self):
#         g_prime = self.calc_g_prime()
        self.dt = self.calc_dt() # Works as long as all ICs are given
        print("dt = ", self.dt) 
        # The order comes from the article
        self.T_1(self.dt) # Water entrainment.
        self.T_2() # TODO: Erosion and deposition
        self.I_1() # Turbidity c. outflows
        self.I_2() # TODO: Update thickness and concentration
        self.I_3() # Update of turbidity flow velocity
        self.I_4() # Toppling rule
                                
        
        
    def T_1(self,dt): # Water entrainment. IN: Q_a,Q_th,Q_cj,Q_v. OUT: Q_vj,Q_th
        '''
        This function calculates the water entrainment.\
        Entrainment is the transport of fluid across an interface\
        between two bodies of fluid by a shear induced turbulent flux.\
        
        '''
        # TODO! Fix correct value of Richardson no. Ri(no flow) = 0 ?
#         ipdb.set_trace()
        g_prime = calc_g_prime(self.Nj, self.Q_cj, self.rho_j, self.rho_a, g = self.g)
        print("g_prime = ", g_prime)
        with np.errstate(divide='ignore',invalid='ignore'):
            Ri = g_prime*self.Q_th/(self.Q_v**2) # Richardson number
        Ri[Ri==0] = np.inf
        E_wStar = 0.075/np.sqrt(1+718*Ri**(2.4)) # Dimensionless incorporation rate
        E_w = self.Q_v*E_wStar # Rate of seawater incorporation
        nQ_th = self.Q_th + E_w*dt # Update cell current thickness
        nQ_th[np.isnan(nQ_th)] = 0
        print("nQ_th =\n", nQ_th, "\nQ_th = \n", self.Q_th)
        with np.errstate(divide='ignore', invalid='ignore'):
            tempQ_cj = self.Q_cj*self.Q_th/nQ_th
        tempQ_cj[np.isnan(tempQ_cj)]=0
        self.Q_cj = tempQ_cj
        print("Q_cj =\n", self.Q_cj)
        
        self.Q_th = nQ_th
        
        
    def T_2(self): # TODO
        '''Erosion and deposition. IN: Q_a,Q_th,Q_cj,Q_cbj,Q_v. OUT: Q_a,Q_d,Q_cj,Q_cbj'''
        pass
    def I_1(self): # Should be done
        '''
        This function calculates the turbidity current outflows.\
        IN: Q_a,Q_th,Q_v,Q_cj. OUT: Q_o
        self.p_f = np.deg2rad(1) # Height threshold friction angle
        
        '''
        # Step (i): angles beta_i 
        g_prime = calc_g_prime(self.Nj, self.Q_cj, self.rho_j, self.rho_a)
        r = self.calc_RunUpHeight(g_prime)
        print("run up height = \n", r)  
        
        q_i = self.Q_a + self.Q_th
        delta = calc_neighborDiff(self.Q_a+r, q_i)
        angle = np.arctan2(delta,self.dx)
        print("angle =\n", np.rad2deg(angle))
        indices = angle>self.p_f # indices(Ny,Nx,6). Dette er basically set A.
        print("indices\n",indices)
       
        
        for ii in range(6): #Step (iii) says to go back to step (ii) if a cell is removed.
            NumberOfCellsInA = np.sum(indices,axis=2) # Cardinality of set A
            print("NumberOfCellsInA =\n",NumberOfCellsInA)
            
            # Step (ii) calculate average
            neighborValues = np.zeros((self.Ny-2,self.Nx-2))        
    #         print("neighbors=\n", self.NEIGHBOR[0])
            for i in range(6):
                neighborValues += q_i[self.NEIGHBOR[i]]*indices[:,:,i] # Vi vil bare legge til verdier hvor angle>self.p_f
            print("neighborValues=\n", neighborValues)    
            with np.errstate(divide='ignore', invalid='ignore'):
                Average = ( (r-self.p_adh)[1:-1,1:-1] + neighborValues)/NumberOfCellsInA
            Average[np.isnan(Average)] = 0 
            print("Average=\n", Average)
            print("indices=\n", indices)

            # Step (iii) Eliminate adjacent cells i with q_i >= Average from A.
            for i in range(6): # Skal sette posisjoner (j) hvor q_i (til nabocelle) > average (i celle j) til 0
                indices[q_i[self.NEIGHBOR[i]]>=Average,i] = 0
        # Step (iv)
        nonNormalizedOutFlow = np.ones((Average.shape + (6,))) * Average[:,:,np.newaxis]
        for i in range(6):
            nonNormalizedOutFlow[:,:,i] -= q_i[self.NEIGHBOR[i]]*indices[:,:,i]
        
        # Step (v)
        with np.errstate(divide='ignore', invalid='ignore'):
            normalization = self.Q_th/r # nu_nf
#         print("normalization=\n", normalization)
        with np.errstate(invalid='ignore'):
            relaxation = np.sqrt(2*r*g_prime/(0.5*self.dx))*self.dt
#         print("relaxation=\n", relaxation)
#         print("(normalization*relaxation)[1:-1,1:-1].shape=", (normalization*relaxation)[1:-1,1:-1].shape)
#         print('nonNormalizedOutFlow.shape=\n', nonNormalizedOutFlow.shape)
        
        for i in range(6):
            self.Q_o[1:-1,1:-1,i] = np.nan_to_num((normalization*relaxation)[1:-1,1:-1] * nonNormalizedOutFlow[:,:,i])
            print("self.Q_o[:,:,",i,"]=\n", self.Q_o[:,:,i])
        
        
    def I_2(self): # TODO 
        '''Update thickness and concentration. IN: Q_th,Q_cj,Q_o. OUT: Q_th,Q_cj'''
        pass
    def I_3(self): # Should be done
        '''
        Update of turbidity flow velocity (speed!). IN: Q_a,Q_th,Q_o,Q_cj. OUT: Q_v.
        '''
#         ipdb.set_trace()
        g_prime = calc_g_prime(self.Nj, self.Q_cj, self.rho_j, self.rho_a)
        print("g_prime I_3 = ", g_prime)
        print("g_prime = ", g_prime)
        sum_q_cj = np.sum(self.Q_cj,axis=2) # TCurrent sediment volume concentration
        print("sum_q_cj = ", sum_q_cj)
        q_o = self.Q_o[1:-1,1:-1]
        print("q_o = ", q_o)
        self.calc_Hdiff()

        U_k = np.zeros((self.Ny-2,self.Nx-2,6))
        diff = self.diff.copy()
        diff[np.isinf(diff)] = 0

        for i in range(6):
            U_k[:,:,i] = np.sqrt(8*g_prime[1:-1,1:-1]*sum_q_cj[1:-1,1:-1]/(self.f*(1+self.a)) * (q_o[:,:,i]*diff[:,:,i]))

        self.Q_v = average_speed_hexagon(U_k)
        
    
    def I_4(self): # Toppling rule
        interiorH = self.Q_d[1:self.Ny-1,1:self.Nx-1]

        angle = np.zeros((self.Ny-2,self.Ny-2,6))
        indices = np.zeros((self.Ny-2,self.Ny-2,6))
        NoOfTrans = np.zeros((self.Ny-2,self.Nx-2))
        frac  = np.zeros((self.Ny-2,self.Nx-2,6))
        deltaS = np.zeros((self.Ny-2,self.Nx-2,6))
        deltaSSum = np.zeros((self.Ny-2,self.Nx-2))

        self.calc_Hdiff()
        diff = self.diff
        
        # Find angles
        dx = self.dx
        angle = ne.evaluate('arctan2(diff,dx)')

        # (Checks if cell (i,j) has angle > repose angle and that it has mass > 0. For all directions.)      
        # Find cells (i,j) for which to transfer mass in the direction given
        for i in np.arange(6):
            indices[:,:,i] = np.logical_and(angle[:,:,i]>self.reposeAngle, (interiorH > 0) )  # Gives indices (i,j) where the current angle > repose angle and where height is > 0


        # Count up the number of cells (i,j) will be transfering mass to. If none, set (i,j) to infinity so that division works.
    #         NoOfTrans = np.sum(indices,axis=2)  # Gir tregere resultat?
        for i in np.arange(6):
            NoOfTrans += indices[:,:,i]
        NoOfTrans[NoOfTrans == 0] = np.inf


        # Calculate fractions of mass to be transfered
        for i in np.arange(6):
            frac[(indices[:,:,i]>0),i] = (0.5 * (diff[(indices[:,:,i]>0),i] - self.dx * np.tan(self.reposeAngle))/(interiorH[(indices[:,:,i]>0)]))
        frac[frac>0.5] = 0.5


        for i in np.arange(6): 
            deltaS[(indices[:,:,i]>0),i] = interiorH[(indices[:,:,i]>0)] * frac[(indices[:,:,i]>0),i]/NoOfTrans[(indices[:,:,i]>0)] # Mass to be transfered from index [i,j] to index [i-1,j]      

        # Lag en endringsmatrise deltaSSum som kan legges til self.Q_d
        # Trekk fra massen som skal sendes ut fra celler
        deltaSSum = -np.sum(deltaS,axis=2)

        # Legg til massen som skal tas imot
        deltaSSum += np.roll(np.roll(deltaS[:,:,0],-1,0),0,1)
        deltaSSum += np.roll(np.roll(deltaS[:,:,1],-1,0),1,1)
        deltaSSum += np.roll(np.roll(deltaS[:,:,2],0,0),1,1) 
        deltaSSum += np.roll(np.roll(deltaS[:,:,3],1,0),0,1) 
        deltaSSum += np.roll(np.roll(deltaS[:,:,4],1,0),-1,1)
        deltaSSum += np.roll(np.roll(deltaS[:,:,5],0,0),-1,1)

        self.Q_d[1:self.Ny-1,1:self.Nx-1] += deltaSSum
        self.totalheight = self.Q_a + self.Q_d # Update total height
        if (self.Q_d < -1e-7).sum() > 0:
            ipdb.set_trace()
            print('height',self.Q_d[1,6])
            raise RuntimeError('Negative sediment thickness!')
            
    def setBathymetry(self, terrain):
        if terrain is not None:
            x = np.linspace(0, 100, self.Nx)
            y = np.linspace(0, 100, self.Ny)
            X = np.array(np.meshgrid(x, y))
            temp = np.zeros((self.Ny,self.Nx))
            if terrain is 'river':
                temp = 2*X[1,:] + 5*np.abs(X[0,:] - 50 + 10*np.sin(X[1,:]/10))
#                 temp = 2*self.X[:,:,1] + 5*np.abs(self.X[:,:,0] + 10*np.sin(self.X[:,:,1]/10))
                self.Q_a +=  temp[::-1,:]  # BRUK MED RIVER
            elif terrain is 'pit':
                temp = np.sqrt((X[0,:]-50)*(X[0,:]-50)+(X[1,:]-50)*(X[1,:]-50))
                self.Q_a += 10*temp
       
        
    def calc_bathymetryDiff(self):
        self.seaBedDiff[:,:,0] = self.Q_a[1:-1,1:-1] - self.Q_a[0:self.Ny-2, 1:self.Nx-1]
        self.seaBedDiff[:,:,1] = self.Q_a[1:-1,1:-1] - self.Q_a[0:self.Ny-2, 2:self.Nx  ]
        self.seaBedDiff[:,:,2] = self.Q_a[1:-1,1:-1] - self.Q_a[1:self.Ny-1, 2:self.Nx  ]
        self.seaBedDiff[:,:,3] = self.Q_a[1:-1,1:-1] - self.Q_a[2:self.Ny  , 1:self.Nx-1]
        self.seaBedDiff[:,:,4] = self.Q_a[1:-1,1:-1] - self.Q_a[2:self.Ny  , 0:self.Nx-2]
        self.seaBedDiff[:,:,5] = self.Q_a[1:-1,1:-1] - self.Q_a[1:self.Ny-1, 0:self.Nx-2]

    def calc_Hdiff(self):
        ''' Calculates the height difference between center cell and neighbors.
            diff[i,j,k] is the '''
        old_height = self.Q_d        
        interiorH = old_height[1:-1,1:-1]
        # Calculate height differences of all neighbors
        self.diff[:,:,0] =  interiorH - old_height[0:self.Ny-2, 1:self.Nx-1] + self.seaBedDiff[:,:,0] 
        self.diff[:,:,1] =  interiorH - old_height[0:self.Ny-2, 2:self.Nx  ] + self.seaBedDiff[:,:,1]
        self.diff[:,:,2] =  interiorH - old_height[1:self.Ny-1, 2:self.Nx  ] + self.seaBedDiff[:,:,2]
        self.diff[:,:,3] =  interiorH - old_height[2:self.Ny  , 1:self.Nx-1] + self.seaBedDiff[:,:,3]
        self.diff[:,:,4] =  interiorH - old_height[2:self.Ny  , 0:self.Nx-2] + self.seaBedDiff[:,:,4]
        self.diff[:,:,5] =  interiorH - old_height[1:self.Ny-1, 0:self.Nx-2] + self.seaBedDiff[:,:,5]
        
    
    def calc_BFroudeNo(self, g_prime): # out: Bulk Froude No matrix
        U = self.Q_v # TODO: Er dette riktig?
        g_prime[g_prime==0] = np.inf 
        return 0.5*U**2/g_prime
    
    def calc_RunUpHeight(self, g_prime): # out: Run up height matrix
        h_k = self.calc_BFroudeNo(g_prime)
        return self.Q_th + h_k
    
    def calc_MaxRelaxationTime(self): #out: matrix
        g_prime = calc_g_prime(self.Nj, self.Q_cj, self.rho_j, self.rho_a, g = self.g)
        r_j = self.calc_RunUpHeight(g_prime)
        r_j[r_j == 0] = np.inf
        g_prime[g_prime==0] = np.inf
        return (self.dx/2)/np.sqrt(2*r_j*g_prime)
    
    def calc_dt(self):
        temp = self.calc_MaxRelaxationTime()
        return np.min(temp[np.nonzero(temp)])
    




In [114]:
# np.seterr(all="raise")
# # Set initial position of mass
x = 1
y = 1
'''
IC: i.e. t = 0
Q_a = bathymetry
Q_th = 0; Q_th(j = source area) = some number
Q_v = 0; Q_v(source) = some number
Q_cj = 0; Q_cj(source) = some number
Q_cbj = fraction of each sediment present in bed
Q_d = thickness of soft sediment, which can be eroded
Q_o = 0
'''
grid = Hexgrid(4,4,reposeAngle = np.deg2rad(0), terrain = None)
# grid.Q_a 
grid.Q_th[y,x] = 1
grid.Q_v[y,x] = 2
grid.Q_cj[y,x] = 1
grid.Q_cbj[y,x] = 1
grid.Q_d[y,x] = 50
# grid.Q_o[y,x] = 
# print(grid.NEIGHBORS[:,:,0])
grid.time_step()
# print(grid.Q_th)
# grid.calc_dt()
# Nt =20
# grid.height[y, x] += 5000
# timeseries = np.zeros(Nt)
# for i in range(Nt):
#     grid.update_heightNew()
#     grid.calc_T_flow_velocity()


# fig = plt.figure(figsize = (9,9))
# ax = fig.add_subplot(111, aspect = 'equal')
# points = ax.scatter(grid.X[:,:,0].flatten(), grid.X[:,:,1].flatten(), marker = 'h', c = grid.seaBed.flatten())
# fig.colorbar(points,fraction=0.026)
# ax.set_title('Terrain(x,y)')

# ax.scatter(grid.X[y, x,0], grid.X[y, x,1]) # Targeting


# len(grid.X[:,:,0].flatten())

dt =  0.102879787301
g_prime =  [[ 0.    0.    0.    0.  ]
 [ 0.    9.81  0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]]
nQ_th =
 [[ 0.          0.          0.          0.        ]
 [ 0.          1.00019624  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]] 
Q_th = 
 [[ 0.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
Q_cj =
 [[[ 0.         0.         0.         0.       ]
  [ 0.         0.         0.         0.       ]
  [ 0.         0.         0.         0.       ]
  [ 0.         0.         0.         0.       ]]

 [[ 0.         0.         0.         0.       ]
  [ 0.         0.9998038  0.         0.       ]
  [ 0.         0.         0.         0.       ]
  [ 0.         0.         0.         0.       ]]

 [[ 0.         0.         0.         0.       ]
  [ 0.         0.         0.         0.       ]
  [ 0.         0.         0.         0.       ]
  [ 0.         0. 

E:\Users\stein\Anaconda3\lib\site-packages\ipykernel_launcher.py:140: RuntimeWarning: invalid value encountered in multiply
E:\Users\stein\Anaconda3\lib\site-packages\ipykernel_launcher.py:154: RuntimeWarning: invalid value encountered in multiply
E:\Users\stein\Anaconda3\lib\site-packages\ipykernel_launcher.py:167: RuntimeWarning: invalid value encountered in multiply


In [75]:
x = np.linspace(0,1,10)
with np.errstate(divide='ignore'):
    np.where(x > 0.0, 1/x, 0)

In [14]:
NW = (np.arange(0,2), np.arange(1,3))

In [60]:
a = np.arange(16).reshape(4,4)
print(a)
shape = a.shape + (3,)
print(shape)
b = np.ones(shape)*a[:,:,np.newaxis]
b[:,:,2]

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
(4, 4, 3)


array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.],
       [ 12.,  13.,  14.,  15.]])

In [73]:
def debug(variable):
    print(variable, '=\n', repr(eval(variable)))

In [77]:
debug('x')

x =
 array([ 0.        ,  0.11111111,  0.22222222,  0.33333333,  0.44444444,
        0.55555556,  0.66666667,  0.77777778,  0.88888889,  1.        ])
